https://adventofcode.com/2021/day/18

In [1]:
#!/usr/bin/env python

# https://adventofcode.com/2021/day/18

import re
from itertools import permutations

with open('data/18.txt') as fh:
    data = fh.read()

    
def find_exploder(s):
    d = 0
    for i, c in enumerate(s):
        if c == '[':
            d += 1
        elif c == ']':
            d -= 1
            if d < 0:
                raise ValueError("negative nesting depth")
        if d == 5:
            m = re.match(r'\[(\d+),(\d+)\]', s[i:])
            if m is not None:
                return (i, m.end(), int(m.group(1)), int(m.group(2)))


def explode(s):
    exploder = find_exploder(s)
    if exploder is None:
        return s
    i, delta, a, b = exploder
    left, right = s[:i], s[i+delta:]

    # Add `a` to rightmost number on the left
    def leftsub(m):
        return str(int(m.group()) + a)
    left = re.sub(r'\d+(?!.*\d)', leftsub, left)
    
    # Add `b` to leftmost number on the right
    def rightsub(m):
        return str(int(m.group()) + b)
    right = re.sub(r'\d+', rightsub, right, count=1)

    return left + '0' + right


def split(s):
    def subfun(m):
        q, r = divmod(int(m.group()), 2)
        return '[' + str(q) + ',' + str(q + r) + ']'
    return re.sub(r'\d\d', subfun, s, count=1)


def reduce(s):
    t = explode(s)
    while t != s:
        s = t
        t = explode(s)
    t = split(s)
    if t == s:
        return t
    else:
        return reduce(t)

    
def add(a, b):
    s = '[' + a + ',' + b + ']'
    return reduce(s)


def magnitude(s):
    def subfun(m):
        return str(int(m.group(1)) * 3 + int(m.group(2)) * 2)
    while '[' in s:
        s = re.sub(r'\[(\d+),(\d+)\]', subfun, s)
    return int(s)


it = iter(data.split())
ttl = next(it)
for n in it:
    ttl = add(ttl, n)
part_1 = magnitude(ttl)
print('part_1 =', part_1)


part_2 = max(magnitude(add(a, b)) for (a, b) in permutations(data.split(), 2))
print('part_2 =', part_2)

part_1 = 3793
part_2 = 4695
